## Create an NWB 2.0 File with Frank Lab data

This notebook shows how to create an [Neurodata Without Borders: Neurophysiology (NWB:N)](https://neurodatawithoutborders.github.io/) file from publicly available Frank Lab data. We will create an NWB file storing one day of data from one animal, inlcluding both awake behaving and sleep epochs, using the [PyNWB](https://pynwb.readthedocs.io/en/latest/) API. You can think of the PyNWB API as a toolbox for working with NWB files in the Python programming language.






### Import dependencies

In [296]:
%load_ext autoreload
%autoreload 2

import pynwb

# General dependencies
import os
import numpy as np
import scipy.interpolate as interpolate

# Time
from datetime import datetime
from dateutil import tz

# Helpers for parsing Frank Lab Matlab data
import franklabnwb.nspike_helpers as ns 

# Frank Lab PyNWB extensions and extension-related helpers
import franklabnwb.fl_extension as fle
import franklabnwb.fl_extension_helpers as flh

# Plotting
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
mdates.rcParams.update({'date.autoformatter.microsecond': '%H:%M:%S.%f'})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Which data do you want to load?

We will create an NWB file storing data from one experimental day for one animal.

In [297]:
# -------
# ~ ~ ~ ~ UPDATE THIS VARIABLE WITH THE PATH TO YOUR DATA ~ ~ ~ ~
# -------
# Path to your data directory (e.g. the directory called "Bon" downloaded from CRCNS)
#data_dir = os.path.expanduser('~/Data/CRCNS/Bon')
data_dir = '/data/mkarlsso/Bon'
nwb_dir = data_dir + '/NWB/'
assert(os.path.isdir(data_dir)), "Data directory does not exist! Update path."

# -------
# You don't need to change these unless you are trying out other days or animals.
# We recommend starting with animal 'Bon', day 4.
# -------
day = 4
animal = 'Bon'

# We don't know the exact date and time this experiment was actually conducted, so we use a placeholder date.
dataset_zero_time = datetime(2006, 1, day, 12, 0, 0, tzinfo=tz.gettz('US/Pacific'))

# Recording parameters needed for import:
eeg_samprate = 1500.0 # Hz

### Initialize the new NWBFile object

The PyNWB API provides a set of easily-used functions that allow us to store our data in the
appropriate places for eventually saving as a valid NWB file.

In [298]:
nwbf = pynwb.NWBFile(
           session_description='Frank Lab CRCNS data for animal {0}, day {1}'.format(animal, day),
           identifier='{0}{1:04}'.format(animal, day),
           session_start_time=dataset_zero_time,
           file_create_date=datetime.now(tz.tzlocal()),
           lab='Frank Laboratory',
           experimenter='Mattias Karlsson',
           institution='UCSF',
           experiment_description='Tetrode recordings from behaving rat on W-Track and during sleep',
           session_id='{0}{1:04}'.format(animal, day))



### Set the subject information

In [299]:
start_time = datetime(1980, 1, 1, 1, tzinfo=tz.gettz('US/Pacific'))

#change when NWB date_of_birth bug fixed
#nwbf.subject = pynwb.file.Subject(date_of_birth=unknown_birth_date,
nwbf.subject = pynwb.file.Subject(description='Long Evans Rat', 
                                  genotype='WT', 
                                  sex='M', 
                                  species='rat', 
                                  subject_id='Bond', 
                                  weight='unknown')

### Process tetrodes metadata

1. Each tetrode gets an [ElectrodeGroup](https://pynwb.readthedocs.io/en/latest/pynwb.ecephys.html#pynwb.ecephys.ElectrodeGroup), where we store metadata about the tetrode such a unique name and the region of the brain it's in. 

2. Each individual recording channel (i.e. each of the 4 tetrode channels) gets its own row in the top-level [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table. Here we store metadata about the electrode such as its location/depth in the brain, and the tetrode that it is part of.  We use the [NWBFile.add_electrode()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_electrode) method to add each channel. As with the NWBFile.epochs table, discussed above, this is a DynamicTable.

3. Each tetrode gets an "Electrode Table Region", an instance of [DynamicTableRegion](https://pynwb.readthedocs.io/en/latest/pynwb.core.html?highlight=DynamicTableRegion#pynwb.core.DynamicTableRegion). This is just a slice into the [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table selecting the channels that go with the tetrode. We use the [create_electrode_table_region()]() method to add each tetrode's Electrode Table Region.

4. Since LFP is often taken from a subset of a tetrode's channels, we create an Electrode Table Region for each tetrode's LFP channles. For example, if LFP was taken from just the first channel of a tetrode, we would create an Electrode Table Region just pointing to that channel of the [NWBFile.electrodes](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.electrodes) table.

In [300]:
# Parse tetrodes metadata from the old Frank Lab Matlab files
tetrode_metadata = ns.parse_franklab_tetrodes(data_dir, animal, day)

# Represent our acquisition system with a 'Device' object
recording_device = nwbf.create_device(name='NSpike acquisition system')

# Four channels per tetrode by definition 
num_chan_per_tetrode = 4     

# Initialize dictionaries to store the metadata
tet_electrode_group = {}  # group for each tetrode
tet_electrode_table_region = {}  # region for each tetrode (all channels)
lfp_electrode_table_region = {}  # region for each tetrode's LFP channels

chan_num = 0   # Incrementing channel number
lfp_channels = list()
for tet_num, tet in tetrode_metadata.items():
    
    # Define some metadata parameters
    tetrode_name = "%02d" % (tet_num) 
    impedance = np.nan
    filtering = 'unknown - likely 600Hz-6KHz'
    location = ns.get_franklab_tet_location(tet)  # area/subarea in the brain
    depth = ns.get_franklab_tet_depth(tet)  # depth in the brain
    description = "tetrode {tet_num} located in {location} on day {day}".format(
        tet_num=tet_num, location=location, day=day)
    
    # 1. Represent the tetrode in NWB as an ElectrodeGroup
    tet_electrode_group[tet_num] = nwbf.create_electrode_group(name=tetrode_name,
                                                               description=description,
                                                               location=location,
                                                               device=recording_device)
    
    # 2. Represent each channels of the tetrode as a row in the NWBFile.electrodes table.
    #    We do not have x and y coordinates for electrodes, so we set to np.nan.
    for i in range(num_chan_per_tetrode):
            nwbf.add_electrode(x=np.nan,  
                               y=np.nan,
                               z=depth,
                               imp=impedance,
                               location=location,
                               filtering=filtering,
                               group=tet_electrode_group[tet_num],  # tetrode this electrode belongs to
                               group_name=tet_electrode_group[tet_num].name,
                               id=chan_num)
            chan_num = chan_num + 1  # total number of channels processed so far across all tets
            
    # 3. Create an Electrode Table Region (slice into the electrodes table) for each tetrode
    table_region_description = 'tetrode %d all channels' % tet_num
    table_region_name = '%d' % tet_num
    table_region_rows = list(range(chan_num - num_chan_per_tetrode, chan_num)) # rows of NWBFile.electrodes table
    tet_electrode_table_region[tet_num] = nwbf.create_electrode_table_region(
        region=table_region_rows,
        description=table_region_description,
        name=table_region_name)

    # 4. Create an ElectrodeTableRegion for all LFP recordings
    lfp_channels.append(chan_num - num_chan_per_tetrode)# Assume that LFP is taken from the first channel
    
table_region_description = 'tetrode LFP channels'  
lfp_electrode_table_region = nwbf.create_electrode_table_region(
    region=lfp_channels,
    description=table_region_description,
    name='electrodes')



### Initialize PyNWB objects for storing behavioral timeseries data

PyNWB provides several datatypes for specific kinds of behavior data. This allows anyone using PyNWB to know what kinds of data are stored in which places in the PyNWB file. However, all of these are examples of timeseries data (i.e. data with assocaited timestamps). Here, we use the following:
- spatial position (x/y) will be stored in a [Position](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.Position) object
- head direction (angle) will be stored in a [CompassDirection](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.CompassDirection) object
- speed (m/s) will be stored in a more general [BehavioralTimeSeries](https://pynwb.readthedocs.io/en/latest/pynwb.behavior.html#pynwb.behavior.BehavioralTimeSeries) object

These objects will later be used to store the specified types of data, but for now they are empty.

In [301]:
position = pynwb.behavior.Position(name='Position')
head_dir = pynwb.behavior.CompassDirection(name='Head Direction')
speed = pynwb.behavior.BehavioralTimeSeries(name='Speed')
linpos = pynwb.behavior.BehavioralTimeSeries(name='Linearized Position')

In [313]:
behavior_data = ns.parse_franklab_behavior_data(data_dir, animal, day)
epoch_time_ivls = []
time_idx, x_idx, y_idx, dir_idx, vel_idx = range(5)  # column ordering of the behavioral data matrix

all_epochs_taskdata = ns.parse_franklab_task_data(data_dir, animal, day)

#load the linear position data
linpos_data = ns.parse_franklab_linpos_data(data_dir, animal, day)

# Initialize empty arrays for behavior samples across all epochs
pos_samples = np.zeros((0, 2)) # x/y positions (n x 2)
dir_samples = np.array([])
speed_samples = np.array([])
behavior_timestamps = []  # behavior timestamps are shared except for linpos
linpos_timestamps = []
apparatus_dict = dict()
statematrix_dict = dict()

# Loop over each epoch of this day
for epoch_num, epoch_data in behavior_data.items():
    m_per_pixel = epoch_data['cmperpixel'][0,0]/100    # meters / pixel conversion factor
    
    
    # Behavior samples for this epoch (position, head direction, speed)
    epoch_pos_samples = epoch_data['data'][:, (x_idx, y_idx)] * m_per_pixel
    epoch_dir_samples = epoch_data['data'][:, dir_idx]
    epoch_speed_samples = epoch_data['data'][:, vel_idx] * m_per_pixel
    
    # Timestamps for this epoch (note that we convert timestamps to POSIX time)
    epoch_timestamps = epoch_data['data'][:, time_idx] + dataset_zero_time.timestamp()
    
    # Add this epoch's data to the array   
    pos_samples = np.concatenate((pos_samples, epoch_pos_samples), axis=0)
    dir_samples = np.concatenate((dir_samples, epoch_dir_samples), axis=0)
    speed_samples = np.concatenate((speed_samples, epoch_speed_samples), axis=0)
    behavior_timestamps = np.concatenate((behavior_timestamps, epoch_timestamps), axis=0)
    
    # Store the times of epoch start and end. We will use these later to build the 'epochs' table
    epoch_time_ivls.append([epoch_timestamps[0], epoch_timestamps[-1]])
    

    # parse the task information structure so we can determine the type of epoch we're in
    task_data = all_epochs_taskdata[epoch_num]
    if task_data['type'][0] == 'sleep':
        if 'Sleep Box' not in apparatus_dict.keys():
            # for sleep epochs we do not create an polygon, so we just create an empty apparatus object
            apparatus_dict['Sleep Box'] = fle.Apparatus(name='Sleep Box'.format(epoch_num), nodes=[], edges=[])
    else:
        apparatus_name = task_data['environment'][0]
        if apparatus_name not in apparatus_dict.keys():
            apparatus_dict[apparatus_name] = flh.get_apparatus_from_linpos(linpos_data[epoch_num], 
                                                                           name=apparatus_name, 
                                                                           conversion=m_per_pixel)
        # for non-sleep box epochs we also need to process the linear position from the statematrix element
        statematrix = linpos_data[epoch_num]['statematrix']
        # for each field in the statematrix we create a separate timeseries object, and here we first
        # concatenate the data. There is probably a better way to do this. 
        for key in list(statematrix.keys()):
            if key not in statematrix_dict:
                statematrix_dict[key] = statematrix[key]
            else:
                np.concatenate((statematrix_dict[key], statematrix[key]), axis=0)
            

In [314]:
# Add the across-epochs behavioral data to the PyNWB objects
# See place_field_with_queries.ipynb for examples of how we query these for specific epochs
position.create_spatial_series(name='Position', 
                               timestamps=behavior_timestamps, 
                               data=pos_samples, 
                               reference_frame='corner of video frame')

head_dir.create_spatial_series(name='Head direction', 
                               timestamps=behavior_timestamps, 
                               data=dir_samples, 
                               reference_frame='0=direction of top of video frame; ' + 
                                   'positive values clockwise (need to confirm this)')

speed.create_timeseries(name='Speed', 
                        timestamps=behavior_timestamps, 
                        data=speed_samples, 
                        unit='m/s', 
                        description='smoothed movement speed estimate')

# add timeseries to linpos
for key in statematrix_dict.keys():
    if key == 'time':
        continue
    else:
        linpos.create_timeseries(name=key, 
                                 timestamps=statematrix_dict['time'],
                                 data = statematrix_dict[key],
                                 description='linear position statematrix information')
        

ValueError: 'Position' already exists in 'Position'

In [304]:
# create a ProcessingModule for behavior data
behav_mod = nwbf.create_processing_module(name='Behavior', 
                                          description='Behavioral data')

# Add the position, head direction and speed data to the ProcessingModule
behav_mod.add_data_interface(position)
behav_mod.add_data_interface(head_dir)
behav_mod.add_data_interface(speed)
behav_mod.add_data_interface(linpos)

# create a ProcessingModule for task data
task_mod = nwbf.create_processing_module(name='Task', 
                                          description='Task data')


# ---------
# Add all three Apparatuses to the "Behavior" ProcessingModule
# ---------
task_mod.add_data_interface(sleep_box_apparatus)
task_mod.add_data_interface(wtrack_A_apparatus)
task_mod.add_data_interface(wtrack_B_apparatus)

print("Note that our fl_extension.Apparatus objects are now in the Task ProcessingModule:")
print(nwbf.modules['Task'])



Linearized Position <class 'pynwb.behavior.BehavioralTimeSeries'>
Fields:
  time_series: { headdir <class 'pynwb.base.TimeSeries'>,  lindist <class 'pynwb.base.TimeSeries'>,  linearDistanceToWells <class 'pynwb.base.TimeSeries'>,  linearVelocity <class 'pynwb.base.TimeSeries'>,  referenceWell <class 'pynwb.base.TimeSeries'>,  segmentHeadDirection <class 'pynwb.base.TimeSeries'>,  segmentIndex <class 'pynwb.base.TimeSeries'>,  traj <class 'pynwb.base.TimeSeries'>,  wellExitEnter <class 'pynwb.base.TimeSeries'> }

### Estimate the apparatus geometries from animal position data
Accurately representing the geometry and topology of behavioral apparatuses (i.e. tracks, mazes, open fields, sleep boxes) is essential for interpreting our spatial data. Here, we estimate track geometry by visually inspecting the animal position records. At the top of this cell, hard-coded values are provided for animal 'Bon', day 4. For other animals and days, you might need to adjust these values based on the plots of position data on each apparatus.  

In the next cell, we will incorporate these geometry coordinates into formal Frank Lab Apparatus objects (franklab.extensions.yaml).

### Store apparatuses as NWB Apparatus objects (Frank Lab extension)

As shown above, we have three apparatuses: Sleep Box, W-track A, and W-track B. We represent each behavioral apparatus as a Frank Lab Apparatus (franklab.extensions.yaml), which uses a graph representation (i.e. nodes and edges) to represent the topology of a track. Each Node represents an important component of the apparatus:
- PointNode represents a point with an x/y position (e.g. reward well, novel object)
- SegmentNode represents a 1D line segment (e.g. linearized maze arm)
- PolygonNode represents a 2D area (e.g. open field / non-linearizable area)

Each Node object has a 'coords' field that describes its spatial geometry, which we found in the previous cell. Nodes sharing at least one coordinate can be represented as spatially connected by adding an Edge. For example, we can represent a reward well (PointNode) at the end of a linearized W-track arm (SegmentNode) by adding an Edge connecting those nodes. 

In [311]:
apparatus_dict

{'Sleep Box': 
 Sleep Box <class 'franklabnwb.fl_extension.Apparatus'>
 Fields:
   edges: { }
   nodes: { }, 'TrackB': 
 TrackB <class 'franklabnwb.fl_extension.Apparatus'>
 Fields:
   edges: { segment0<->segment1 <class 'franklabnwb.fl_extension.Edge'>,  segment0<->segment3 <class 'franklabnwb.fl_extension.Edge'>,  segment0<->well0 <class 'franklabnwb.fl_extension.Edge'>,  segment1<->segment2 <class 'franklabnwb.fl_extension.Edge'>,  segment1<->segment3 <class 'franklabnwb.fl_extension.Edge'>,  segment2<->well1 <class 'franklabnwb.fl_extension.Edge'>,  segment3<->segment4 <class 'franklabnwb.fl_extension.Edge'>,  segment4<->well2 <class 'franklabnwb.fl_extension.Edge'> }
   nodes: { segment0 <class 'franklabnwb.fl_extension.SegmentNode'>,  segment1 <class 'franklabnwb.fl_extension.SegmentNode'>,  segment2 <class 'franklabnwb.fl_extension.SegmentNode'>,  segment3 <class 'franklabnwb.fl_extension.SegmentNode'>,  segment4 <class 'franklabnwb.fl_extension.SegmentNode'>,  well0 <class 'fra

In [315]:
   
# load the task information
all_epochs_taskdata = ns.parse_franklab_task_data(data_dir, animal, day)
apparatus_dict = dict()
for epoch_num in all_epochs_taskdata.keys():
    
    # parse the task information structure
    task_data = all_epochs_taskdata[epoch_num]

    # create the necessary apparati. 
    # NOTE that in reality the same track has slightly different linear coordinates in each epoch, 
    # but for simplicity we only take the one from the first epoch for each track. As we are also saving the
    # linearized position this should be okay.
    if task_data['type'][0] == 'sleep':
        if 'Sleep Box' not in apparatus_dict.keys():
            # for sleep epochs we do not create an polygon, so we just create an empty apparatus object
            apparatus_dict['Sleep Box'] = fle.Apparatus(name='Sleep Box'.format(epoch_num), nodes=[], edges=[])
    else:
        apparatus_name = task_data['environment'][0]
        if apparatus_name not in apparatus_dict.keys():
            apparatus_dict[apparatus_name] = flh.get_apparatus_from_linpos(linpos_data[epoch_num], 
                                                                           name=apparatus_name, 
                                                                           conversion=m_per_pixel)

### Store the Apparatuses in the NWBFile object

After building the Apparatus objects, we store them in the "Behavior" [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule), just like we did with the other behavioral data above.

In [316]:
# create a ProcessingModule for apparatus data
apparatus_mod = nwbf.create_processing_module(name='Apparatus', 
                                          description='Apparatus data')

task_mod = nwbf.create_processing_module(name='Task', 
                                          description='Task data')
# ---------
# Add all  Apparatuses to the "Behavior" ProcessingModule
# ---------
for apparatus in apparatus_dict:
    task_mod.add_data_interface(apparatus_dict[apparatus])


### Represent behavioral tasks using Frank Lab NWB extension
We also represent each behavioral task that the animal may perform on an apparatus as a Frank Lab Task (franklab.extensions.yaml). This object simply contains a name and a detailed description of the task. For the dataset here, we only have two tasks: W-Alternation and Sleep.

We then store Task objects in the "Behavior" [ProcessingModule](https://pynwb.readthedocs.io/en/latest/pynwb.base.html#pynwb.base.ProcessingModule), just like we did with the other behavioral data above.

In [317]:
task_name = 'Sleep'
description = 'The animal sleeps or wanders freely around a small, empty box.'
task_mod.add_data_interface(fle.Task(name=task_name, description=description))

task_name = 'W-Alternation'
task_description = 'The animal runs in an alternating W pattern between three neighboring arms of a maze.'
task_mod.add_data_interface(fle.Task(name=task_name, description=task_description))

print("Note that we've added fl_extension.Task objects to our ProcessingModule:")
print(nwbf.modules['Task'])
print(nwbf.modules['Task']['W-Alternation'])
print(nwbf.modules['Task']['Sleep'])

Note that we've added fl_extension.Task objects to our ProcessingModule:

Task <class 'pynwb.base.ProcessingModule'>
Fields:
  data_interfaces: { Sleep <class 'franklabnwb.fl_extension.Task'>,  Sleep Box <class 'franklabnwb.fl_extension.Apparatus'>,  TrackA <class 'franklabnwb.fl_extension.Apparatus'>,  TrackB <class 'franklabnwb.fl_extension.Apparatus'>,  W-Alternation <class 'franklabnwb.fl_extension.Task'> }
  description: Task data


W-Alternation <class 'franklabnwb.fl_extension.Task'>
Fields:
  description: The animal runs in an alternating W pattern between three neighboring arms of a maze.


Sleep <class 'franklabnwb.fl_extension.Task'>
Fields:
  description: The animal sleeps or wanders freely around a small, empty box.



### Store epoch metadata in the NWBFile object
We store information about different sections of a day's worth of experiments as Epochs in the top-level [NWBFile.epochs](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.epochs) table. By default, there are required columns for 'start_time', 'stop_time', and 'tags'. We add additional metadata columns for the epoch's Task, Apparatus, etc. using the [NWBFile.add_epoch_column()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_epoch_column) method. After we have all of the metadata columns set up, we add each epoch as a new row of the table using the [NWBFile.add_epoch()](https://pynwb.readthedocs.io/en/latest/pynwb.file.html#pynwb.file.NWBFile.add_epoch) method.

<i>Take a look under the PyNWB hood</i>:</br>
Each epoch occurs in a discrete time interval defined by its start and stop times. As such, the [NWBFile.epochs](https://pynwb.readthedocs.io/en/latest/pynwb.file.html?highlight=epochs#pynwb.file.NWBFile.epochs) table is an instance of [TimeIntervals](https://pynwb.readthedocs.io/en/latest/pynwb.epoch.html?highlight=TimeIntervals#pynwb.epoch.TimeIntervals), which is itself an instance of [DynamicTable](https://pynwb.readthedocs.io/en/latest/pynwb.core.html#pynwb.core.DynamicTable). Later, we will store electrodes and clustered units in two other DynamicTables. One of the advantages of using DynamicTables is it allows for adding arbitrary columns without having to write an extension.

In [159]:
# ---------
# Load epochs metadata from the Frank Lab Matlab files
# ---------
all_epochs_metadata = ns.parse_franklab_task_data(data_dir, animal, day)

# ---------
# Add metadata columns to the NWBFile.epochs table
# By default, it has columns for 'start_time', 'stop_time', and 'tags'.
# ---------
nwbf.add_epoch_column(name='exposure', description='number of exposures to this apparatus')
nwbf.add_epoch_column(name='task', description='behavioral task for this epoch')
nwbf.add_epoch_column(name='apparatus', description='behavioral apparatus for this epoch')

# ---------
# Iteratively add each epoch to the NWBFile.epochs table
# ---------
task_exposure_dict = dict()
no_exposure_start_num = 1000000
for epoch_num, epoch_metadata in all_epochs_metadata.items():
    
    # start and stop times were inferred from the behavior data earlier
    epoch_start_time, epoch_stop_time = epoch_time_ivls[epoch_num-1]  
    
    # meter per pixel ratio is also in the behavior data
    m_per_pixel = behavior_data[epoch_num]['cmperpixel'][0,0]/100  
    
    # Frank Lab Task (from the "Task" ProcessingModule)
    epoch_task = flh.get_franklab_task(epoch_metadata, task_mod)
    
    # Frank Lab Apparatus (from the "Task" ProcessingModule)
    epoch_apparatus = flh.get_franklab_apparatus(epoch_metadata, task_mod)
    
    epoch_exposure_num = ns.get_exposure_num(epoch_metadata)
    # if this is -1then we'll replace it with something that starts at 1e6 so it's always a number
    if epoch_exposure_num == -1:
        # check to see if this is the first time that this task 
        if epoch_task in task_exposure_dict.keys():
            task_exposure_dict[epoch_task] += 1
            exposure = task_exposure_dict[epoch_task]
        else:
            task_exposure_dict[epoch_task] = no_exposure_start_num
    # Required column 'tags'. We do not presently use this.
    epoch_tags = ''
    
    # Add this epoch to the NWBFile.epochs table
    # Note that task and apparatus are references to the "Behavior" ProcessingModule, 
    # so they will not be unnecessarily duplicated within the NWBFile
    nwbf.add_epoch(start_time=epoch_start_time,
                   stop_time=epoch_stop_time,
                   exposure=epoch_exposure_num,
                   task=epoch_task,
                   apparatus=epoch_apparatus,
                   tags=epoch_tags)

ValueError: column 'exposure' already exists in DynamicTable 'epochs'

In [20]:
print("Here is an example epoch from the table.\nNote that the 'task' and 'apparatus' columns point to our Frank Lab extension objects.\n")
print(nwbf.epochs.to_dataframe().iloc[0, :])

Here is an example epoch from the table.
Note that the 'task' and 'apparatus' columns point to our Frank Lab extension objects.

start_time                                          1.13641e+09
stop_time                                           1.13641e+09
exposure                                                     NA
task          \nSleep <class 'franklabnwb.fl_extension.Task'...
apparatus     \nSleep Box <class 'franklabnwb.fl_extension.A...
tags                                                         []
Name: 0, dtype: object


### Inspect the NWBFile
We have now added many of the core features of Frank Lab data to the NWBFile. Note that we have not used all of the available top-level fields ('analysis', 'stimulus', etc). These fields can be used to store other kinds of data as necessary for your lab and analysis pipeline. However, it is not advised to store temporary or in-progress analyses in the NWBFile, as NWB is meant to store stable versions of the data.

In [21]:
print(nwbf)


root <class 'pynwb.file.NWBFile'>
Fields:
  acquisition: { LFP <class 'pynwb.ecephys.LFP'> }
  analysis: { }
  devices: { NSpike acquisition system <class 'pynwb.device.Device'> }
  electrode_groups: { 01 <class 'pynwb.ecephys.ElectrodeGroup'>,  02 <class 'pynwb.ecephys.ElectrodeGroup'>,  03 <class 'pynwb.ecephys.ElectrodeGroup'>,  04 <class 'pynwb.ecephys.ElectrodeGroup'>,  05 <class 'pynwb.ecephys.ElectrodeGroup'>,  06 <class 'pynwb.ecephys.ElectrodeGroup'>,  07 <class 'pynwb.ecephys.ElectrodeGroup'>,  08 <class 'pynwb.ecephys.ElectrodeGroup'>,  10 <class 'pynwb.ecephys.ElectrodeGroup'>,  11 <class 'pynwb.ecephys.ElectrodeGroup'>,  12 <class 'pynwb.ecephys.ElectrodeGroup'>,  13 <class 'pynwb.ecephys.ElectrodeGroup'>,  14 <class 'pynwb.ecephys.ElectrodeGroup'>,  15 <class 'pynwb.ecephys.ElectrodeGroup'>,  17 <class 'pynwb.ecephys.ElectrodeGroup'>,  18 <class 'pynwb.ecephys.ElectrodeGroup'>,  19 <class 'pynwb.ecephys.ElectrodeGroup'>,  20 <class 'pynwb.ecephys.ElectrodeGroup'>,  21 <c

### Write the NWBFile to disk

In [22]:
nwb_filename = "{0}{1}{2:02}.nwb".format(nwb_dir, animal.lower(), day)
with pynwb.NWBHDF5IO(nwb_filename, mode='w') as iow:
    iow.write(nwbf)
print('Successfully wrote NWB file: ' + nwb_filename)

Successfully wrote NWB file: /data/mkarlsso/Bon/NWB/bon04.nwb


### Read our NWBFile from disk to test round-trip

In [319]:
with pynwb.NWBHDF5IO(nwb_filename, mode='r') as ior:
    nwbf_read = ior.read()
    print(nwbf_read)


root <class 'pynwb.file.NWBFile'>
Fields:
  acquisition: { LFP <class 'pynwb.ecephys.LFP'> }
  analysis: { }
  devices: { NSpike acquisition system <class 'pynwb.device.Device'> }
  electrode_groups: { 01 <class 'pynwb.ecephys.ElectrodeGroup'>,  02 <class 'pynwb.ecephys.ElectrodeGroup'>,  03 <class 'pynwb.ecephys.ElectrodeGroup'>,  04 <class 'pynwb.ecephys.ElectrodeGroup'>,  05 <class 'pynwb.ecephys.ElectrodeGroup'>,  06 <class 'pynwb.ecephys.ElectrodeGroup'>,  07 <class 'pynwb.ecephys.ElectrodeGroup'>,  08 <class 'pynwb.ecephys.ElectrodeGroup'>,  10 <class 'pynwb.ecephys.ElectrodeGroup'>,  11 <class 'pynwb.ecephys.ElectrodeGroup'>,  12 <class 'pynwb.ecephys.ElectrodeGroup'>,  13 <class 'pynwb.ecephys.ElectrodeGroup'>,  14 <class 'pynwb.ecephys.ElectrodeGroup'>,  15 <class 'pynwb.ecephys.ElectrodeGroup'>,  17 <class 'pynwb.ecephys.ElectrodeGroup'>,  18 <class 'pynwb.ecephys.ElectrodeGroup'>,  19 <class 'pynwb.ecephys.ElectrodeGroup'>,  20 <class 'pynwb.ecephys.ElectrodeGroup'>,  21 <c

In [43]:
nwb_filename

'/data/mkarlsso/Bon/NWB/bon04.nwb'

In [322]:
pos = behav_mod.data_interfaces['Position']

In [324]:
pos.spatial_series

{'Position': 
 Position <class 'pynwb.behavior.SpatialSeries'>
 Fields:
   comments: no comments
   conversion: 1.0
   data: [[0.3931875 0.351    ]
  [0.3965625 0.3493125]
  [0.3965625 0.3493125]
  ...
  [0.4303125 0.3645   ]
  [0.4303125 0.3628125]
  [0.4303125 0.3645   ]]
   description: no description
   interval: 1
   num_samples: 214497
   reference_frame: corner of video frame
   resolution: 0.0
   timestamps: [1.13640581e+09 1.13640581e+09 1.13640581e+09 ... 1.13641415e+09
  1.13641415e+09 1.13641415e+09]
   timestamps_unit: Seconds
   unit: meters}

In [62]:
          epochs = nwbf.epochs.to_dataframe()


In [91]:
%config IPCompleter.greedy=True

In [131]:
          for enum in range(len(epochs)):
               print(type(epochs['exposure'][enum]))

<class 'str'>
<class 'numpy.uint8'>
<class 'str'>
<class 'numpy.uint8'>
<class 'str'>
<class 'numpy.uint8'>
<class 'str'>


In [139]:
testdict = dict()
testdict[task_mod] = task_mod

In [142]:
testdict[task_mod]


Task <class 'pynwb.base.ProcessingModule'>
Fields:
  data_interfaces: { Sleep <class 'franklabnwb.fl_extension.Task'>,  Sleep Box <class 'franklabnwb.fl_extension.Apparatus'>,  W-Alternation <class 'franklabnwb.fl_extension.Task'>,  W-track A <class 'franklabnwb.fl_extension.Apparatus'>,  W-track B <class 'franklabnwb.fl_extension.Apparatus'> }
  description: Task data

In [338]:
z = np.empty((0,2))

In [326]:
a = np.asarray([1, 2])

In [327]:
a.shape

(2,)

In [334]:
b = np.asarray([2 , 3])
d = np.asarray([4,5])

In [339]:
c = np.stack((z,a,b,d))

ValueError: all input arrays must have the same shape

In [337]:
c.shape

(3, 2)